In [157]:
import pandas as pd
import seaborn as sns
import numpy as np
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
from sklearn.preprocessing import Imputer

In [95]:
faults=pd.read_csv('data/J1939Faults.csv')

In [133]:
faults.head()

,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,faultValue,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp
0,1,990349,2015-02-21 10:47:13.000,Low (Severity Low) Engine Coolant Level,NaN,unknown,unknown,unknown,unknown,0,111,17,True,2,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:34:25.000
1,2,990360,2015-02-21 11:34:34.000,NaN,NaN,unknown,unknown,unknown,unknown,11,629,12,True,127,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:35:10.000
2,3,990364,2015-02-21 11:35:31.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,False,127,NaN,1369,105336226,41.421250,-87.767361,2015-02-21 11:35:26.000
3,4,990370,2015-02-21 11:35:33.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,True,127,NaN,1369,105336226,41.421018,-87.767361,2015-02-21 11:36:08.000
4,5,990416,2015-02-21 11:39:41.000,NaN,NaN,22281684P01*22357957P01*22362082P01*,13063430,0USA13_13_0415_2238A,VOLVO,0,4364,17,False,2,NaN,1674,105427130,38.416481,-89.442638,2015-02-21 11:39:37.000


In [97]:
faults.shape

(1187335, 20)

In [98]:
# vdod=vehicle diagnostic onboard data
vdod=pd.read_csv('data/VehicleDiagnosticOnboardData.csv')

In [99]:
vdod.head()

,Id,Name,Value,FaultId
0,1,IgnStatus,False,1
1,2,EngineOilPressure,0,1
2,3,EngineOilTemperature,96.74375,1
3,4,TurboBoostPressure,0,1
4,5,EngineLoad,11,1


In [100]:
vdod.shape

(12821626, 4)

In [101]:
#vdod=vdod.rename(columns={'Name':'fault_nam})

In [102]:
vdod.Name.value_counts()

LampStatus                   1187335
IgnStatus                     608454
EngineRpm                     586921
IntakeManifoldTemperature     586291
EngineOilPressure             586244
EngineCoolantTemperature      586071
BarometricPressure            585976
DistanceLtd                   585819
EngineLoad                    585621
FuelRate                      585237
FuelLtd                       585195
Speed                         583916
EngineOilTemperature          583912
TurboBoostPressure            583351
EngineTimeLtd                 581366
CruiseControlSetSpeed         576458
CruiseControlActive           574916
AcceleratorPedal              531889
FuelLevel                     502795
Throttle                      420503
ParkingBrake                  399972
FuelTemperature               299110
SwitchedBatteryVoltage        114059
ServiceDistance                  215
Name: Name, dtype: int64

In [103]:
# Drop id columns and Pivot the vehicle diagnostics data
vdod_p=vdod.drop(columns='Id', axis=1)
vdod_p=vdod_p.pivot(index='FaultId', columns='Name', values='Value').reset_index()

In [104]:
vdod_p.columns.name = " "

In [105]:
vdod_p.head()

,FaultId,AcceleratorPedal,BarometricPressure,CruiseControlActive,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,EngineTimeLtd,FuelLevel,FuelLtd,FuelRate,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
0,1,0,14.21,False,66.48672,423178.7,100.4,11,0,96.74375,0,1632.2,43.2,12300.907429328,0,NaN,False,78.8,1023,True,NaN,0,3276.75,NaN,0
1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
3,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
4,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16639,NaN,NaN,NaN,NaN,NaN,NaN


In [106]:
vdod_p.shape

(1187335, 25)

In [107]:
faults_vdod=pd.merge(faults, vdod_p, how='inner', left_on='RecordID', right_on='FaultId')

In [140]:
faults_vdod.head()

,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,faultValue,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp,FaultId,AcceleratorPedal,BarometricPressure,CruiseControlActive,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,EngineTimeLtd,FuelLevel,FuelLtd,FuelRate,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,lat_r,lon_r
0,1,990349,2015-02-21 10:47:13.000,Low (Severity Low) Engine Coolant Level,NaN,unknown,unknown,unknown,unknown,0,111,17,True,2,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:34:25.000,1,0,14.21,False,66.48672,423178.7,100.4,11,0,96.74375,0,1632.2,43.2,12300.907429328,0,NaN,False,78.8,1023,True,NaN,0,3276.75,NaN,0,38.9,-84.6
1,2,990360,2015-02-21 11:34:34.000,NaN,NaN,unknown,unknown,unknown,unknown,11,629,12,True,127,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:35:10.000,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN,38.9,-84.6
3,4,990370,2015-02-21 11:35:33.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,True,127,NaN,1369,105336226,41.421018,-87.767361,2015-02-21 11:36:08.000,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN,41.4,-87.8
5,6,990431,2015-02-21 11:40:22.000,Low (Severity Low) Engine Coolant Level,NaN,04993120*00025921*082113134117*07700053*I0*BBZ*,79466580,6X1u10D1500000000,CMMNS,0,111,17,True,1,NaN,1417,105438630,33.043564,-96.179722,2015-02-21 11:40:59.000,6,48,14.4275,False,64.6226,470381.4,181.4,30,38.28,196.5313,1514.5,9480,44,70349.809963756,4.583399,NaN,True,111.2,1023,NaN,NaN,13.6022,3276.75,NaN,6.67,33.0,-96.2
7,8,990441,2015-02-21 11:40:22.000,Low (Severity Low) Engine Coolant Level,NaN,04993120*00022630*082113134117*07700053*I0*BBZ*,79463845,6X1u10D1500000000,CMMNS,0,111,17,True,1,NaN,1429,105356054,38.228796,-84.582500,2015-02-21 11:41:44.000,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,1023,NaN,NaN,NaN,NaN,NaN,NaN,38.2,-84.6


In [109]:
faults_vdod.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1187335 entries, 0 to 1187334
Data columns (total 45 columns):
RecordID                     1187335 non-null int64
ESS_Id                       1187335 non-null int64
EventTimeStamp               1187335 non-null object
eventDescription             1126490 non-null object
actionDescription            0 non-null float64
ecuSoftwareVersion           891285 non-null object
ecuSerialNumber              844318 non-null object
ecuModel                     1122577 non-null object
ecuMake                      1122577 non-null object
ecuSource                    1187335 non-null int64
spn                          1187335 non-null int64
fmi                          1187335 non-null int64
active                       1187335 non-null bool
activeTransitionCount        1187335 non-null int64
faultValue                   0 non-null float64
EquipmentID                  1187335 non-null object
MCTNumber                    1187335 non-null int64
Latitud

In [110]:
#mask = np.column_stack([faults_vdod[col].str.contains(r"\,", na=False) for col in faults_vdod])
#faults_vdod.loc[mask.any(axis=1)]

In [ ]:
faults_vdod_backup=faults_vdod.drop(columns=['RecordID', 'ESS_Id', 'actionDescription', 'faultValue', 'lat_r', \
                                             'lon_r', 'Latitude', 'Longitude')

In [111]:
#round the location of the 3 locations to be removed (they were service centers)
#193 Aldi Blvd, Mt Juliet, TN: 36.0666667, -86.4347222
#137 Eady Road – Bell Buckle, TN: 35.5883333, -86.4438888
#5754 Fish Hatchery Road – Russellville, TN: 36.1950, -83.174722
faults_vdod['lat_r']=faults_vdod['Latitude'].round(decimals = 1)
faults_vdod['lon_r']=faults_vdod['Longitude'].round(decimals=1)


In [112]:
# Remove the locations that are rounded based on the above locations 
faults_vdod=faults_vdod[(faults_vdod['lat_r']!=36.1) & (faults_vdod['lon_r']!= -86.4)]
faults_vdod=faults_vdod[(faults_vdod['lat_r']!=35.6) & (faults_vdod['lon_r']!= -86.4)]
faults_vdod=faults_vdod[(faults_vdod['lat_r']!=36.2) & (faults_vdod['lon_r']!= -83.2)]
#df[df.name != 'Tina']

In [118]:
#Remove active = False
faults_vdod=faults_vdod[faults_vdod['active'] == True]

In [138]:
faults_vdod=faults_vdod[faults_vdod['EquipmentID'].astype(str).str.len()<=5]
faults_vdod.shape

(485141, 47)

In [143]:
faults_vdod=faults_vdod.drop(columns=['RecordID', 'ESS_Id', 'actionDescription', 'ecuSoftwareVersion' , \
                                      'ecuSerialNumber', 'ecuModel', 'ecuMake', 'ecuSource', 'faultValue', \
                                      'MCTNumber', 'Latitude', 'Longitude', 'lat_r', 'lon_r'])

In [150]:
faults_vdod=faults_vdod.drop(columns=['LocationTimeStamp'])

In [156]:
faults_vdod.head(5)

,EventTimeStamp,eventDescription,spn,fmi,active,activeTransitionCount,EquipmentID,FaultId,AcceleratorPedal,BarometricPressure,CruiseControlActive,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,EngineTimeLtd,FuelLevel,FuelLtd,FuelRate,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
0,2015-02-21 10:47:13,Low (Severity Low) Engine Coolant Level,111,17,True,2,1439,1,0,14.21,False,66.48672,423178.7,100.4,11,0,96.74375,0,1632.2,43.2,12300.907429328,0,NaN,False,78.8,1023,True,NaN,0,3276.75,NaN,0
1,2015-02-21 11:34:34,NaN,629,12,True,127,1439,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
3,2015-02-21 11:35:33,Incorrect Data Steering Wheel Angle,1807,2,True,127,1369,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
5,2015-02-21 11:40:22,Low (Severity Low) Engine Coolant Level,111,17,True,1,1417,6,48,14.4275,False,64.6226,470381.4,181.4,30,38.28,196.5313,1514.5,9480,44,70349.809963756,4.583399,NaN,True,111.2,1023,NaN,NaN,13.6022,3276.75,NaN,6.67
7,2015-02-21 11:40:22,Low (Severity Low) Engine Coolant Level,111,17,True,1,1429,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,1023,NaN,NaN,NaN,NaN,NaN,NaN


In [152]:
faults_vdod.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 485141 entries, 0 to 1187333
Data columns (total 32 columns):
EventTimeStamp               485141 non-null datetime64[ns]
eventDescription             459450 non-null object
spn                          485141 non-null int64
fmi                          485141 non-null int64
active                       485141 non-null bool
activeTransitionCount        485141 non-null int64
EquipmentID                  485141 non-null object
FaultId                      485141 non-null int64
AcceleratorPedal             424894 non-null object
BarometricPressure           469604 non-null object
CruiseControlActive          460680 non-null object
CruiseControlSetSpeed        461638 non-null object
DistanceLtd                  469449 non-null object
EngineCoolantTemperature     469596 non-null object
EngineLoad                   469185 non-null object
EngineOilPressure            469701 non-null object
EngineOilTemperature         468230 non-null object
En

In [148]:
#change the data types
# to date time
faults_vdod['EventTimeStamp'] = faults_vdod['EventTimeStamp'].astype('datetime64[ns]')

In [ ]:
#fill nan as 

In [154]:
#Change to str type
faults_vdod['EquipmentID'] = faults_vdod['EquipmentID'].astype(str)
#df[['A', 'C']] = df[['A', 'C']].apply(pd.to_numeric) 
#data["Salary"]= data["Salary"].astype(int) 

In [158]:
# change to int type
#faults_vdod['AcceleratorPedal']=faults_vdod['AcceleratorPedal'].astype(int)
#imp = Imputer(missing_values='NaN', strategy='most_frequent', axis=0)

In [ ]:
#use fillna with 0 for the fault codes values for the initial prediction using RandomForest
faults_vdod[['AcceleratorPedal', 'BarometricPressure', 'CruiseControlSetSpeed', 'DistanceLtd', \
             'EngineCoolantTemperature', ]]
